In [68]:
from bot import get_chroma_db, load_documents
from utils import DocsJSONLLoader, get_openai_api_key, get_file_path
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA, StuffDocumentsChain

In [7]:
import os
OPENAI_API_KEY = 
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [31]:
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
documents = load_documents(get_file_path())
print("Cargando embeddings")
vector_store = get_chroma_db(
    embeddings,
    documents,
    "../chroma_docs"
)
retriever = vector_store.as_retriever(
    search_kwargs = {"k":3}
)
llm = ChatOpenAI(
        model_name = "gpt-3.5-turbo",
        temperature = 0.2,
        max_tokens = 1000
)

Cargando embeddings


In [50]:
memory = ConversationBufferMemory(
    output_key='answer',
    memory_key='chat_history',
    return_messages=True
)

In [120]:
llm_summarizer = OpenAI(
    model="gpt-3.5-turbo-instruct",
    max_tokens=1000,
    temperature=0.0
)

In [119]:
summary_template = """You are a helpful assistant created to summarize conversations and text. Following there is a conversation and a context, 
you have to summarize the conversation in Spanish with a good summary and also summarize the context in Spanish and return a type of JSON response as
chat_summary: your_chat_summary, context_summary:your_context_summary.
    # Conversation: {messages}

    # Context {context}
    
    # Your JSON response between curly brackets:
    """

SUMMARY_TEMPLATE = PromptTemplate(
    template=summary_template,
    input_variables=["messages", "context"]
)

summarizer_chain = LLMChain(llm=llm_summarizer, prompt=SUMMARY_TEMPLATE)

In [100]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

template = """Based on the context provided, provide an answer to the best of your knowledge.
Use your skills to determine what kind of context is provided and tailor your response accordingly. 
When providing an answer, choose the tone of voice and humor of Zapp Brannigan from Futurama. Also, use html bullet list format when needed.
Question: {question}
=========
{context}
=========
"""
QA_PROMPT = PromptTemplate(template=template, input_variables=["question", "context"])

In [101]:
retrieved = ""
documents = retriever.get_relevant_documents("Que son transformers?")
for document in documents:
    to_insert = "- " + document.page_content + '\n'
    retrieved += to_insert

print(retrieved)

- the world. >Since their introduction in 2017, Transformers have rapidly become the state-of-the-art approach to tackle tasks in many domains such as natural language processing, speech recognition, and computer vision. In short, if you’re doing deep learning, then you need Transformers! Lewis Tunstall, Hugging Face ML Engineer & Author of Natural Language Processing with Transformers Timeline of popular Transformer model releases: Source #### 2.4.1 How do Transformers work? Transformers work by leveraging attention, a powerful deep-learning algorithm, first seen in computer vision models. —Not all that different from how we humans process information through attention. We are incredibly good at forgetting/ignoring mundane daily inputs that don’t pose a threat or require a response from us. For example, can you remember everything you saw and heard coming home last Tuesday? Of course not! Our brain’s memory is limited and valuable. Our recall is aided by our ability to forget trivial 

In [125]:
results = summarizer_chain.predict(messages ="AI: Hola, estoy para ayudarte \nUser: Quiero ayuda para una compra \nAI: Claro que si", context = retrieved)


In [126]:
print(results)


{
    "chat_summary": "AI está aquí para ayudar al usuario con una compra.",
    "context_summary": "Transformers es una herramienta de aprendizaje automático de vanguardia que se utiliza en una variedad de tareas en diferentes dominios, como el procesamiento del lenguaje natural, el reconocimiento de voz y la visión por computadora. Los Transformers funcionan mediante el uso de la atención, un algoritmo de aprendizaje profundo poderoso, y proporcionan una forma eficiente de entrenar modelos preentrenados para tareas comunes en diferentes modalidades. También ofrecen interoperabilidad entre diferentes frameworks y soporte para modelos personalizados." 
}


In [127]:
conversation = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever = retriever,
    verbose = True
)

In [134]:
chat_history = []
query = input("Entre su pregunta: ")
result = conversation({"question":query, "chat_history":chat_history})
chat_history.append((query, result["answer"]))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
the world. >Since their introduction in 2017, Transformers have rapidly become the state-of-the-art approach to tackle tasks in many domains such as natural language processing, speech recognition, and computer vision. In short, if you’re doing deep learning, then you need Transformers! Lewis Tunstall, Hugging Face ML Engineer & Author of Natural Language Processing with Transformers Timeline of popular Transformer model releases: Source #### 2.4.1 How do Transformers work? Transformers work by leveraging attention, a powerful deep-learning algorithm, first seen in computer vision models. —Not all that different from how we humans process information through attention. We are incredibly good at forge

In [138]:
query = input("Entre su pregunta: ")
result = conversation({"question":query, "chat_history":chat_history})
chat_history.append((query, result["answer"]))



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Que son los transformers?
Assistant: Los Transformers son un enfoque de vanguardia en el campo del aprendizaje profundo que se utiliza para abordar tareas en diversos dominios, como el procesamiento del lenguaje natural, el reconocimiento de voz y la visión por computadora. Son modelos de inteligencia artificial que utilizan un algoritmo llamado atención para procesar la información de manera eficiente. Los Transformers son ampliamente utilizados en aplicaciones de deep learning y han demostrado ser muy efectivos en una variedad de tareas.
Human: Como los podria cargar y usar?
Assistant: La forma de cargar y utilizar los Transformers es a través del uso de la biblioteca Transformers de Hugging Face. Puedes seguir una guía rápida para comenzar y aprender cómo us

In [136]:
results

'\n{\n    "chat_summary": "AI está aquí para ayudar al usuario con una compra.",\n    "context_summary": "Transformers es una herramienta de aprendizaje automático de vanguardia que se utiliza en una variedad de tareas en diferentes dominios, como el procesamiento del lenguaje natural, el reconocimiento de voz y la visión por computadora. Los Transformers funcionan mediante el uso de la atención, un algoritmo de aprendizaje profundo poderoso, y proporcionan una forma eficiente de entrenar modelos preentrenados para tareas comunes en diferentes modalidades. También ofrecen interoperabilidad entre diferentes frameworks y soporte para modelos personalizados." \n}'